In [1]:
import time
import copy
import math
import torch

from nslug.main_gp import gp  
from nslug.main_nslug import nslug  
from nslug.main_ga import ga
from nslug.datasets.data_loader import load_parkinson , load_gametes
from nslug.evaluators.fitness_functions import accuracy  
from nslug.utils.utils import (protected_div,get_terminals,get_best_max,create_custom_list,train_test_split)
from nslug.initializers.initializers import *

from nslug.algorithms.GP.representations.tree_utils import create_grow_random_tree,create_full_random_tree,flatten,used_terminals,fix_terminals,fix_tree,create_random_random_tree,create_full_random_tree_list_mode
from nslug.algorithms.GP.representations.tree import Tree
from nslug.algorithms.GP.operators.mutators import *


from nslug.algorithms.GA.representations.population import Population_ga
from nslug.algorithms.GA.representations.individual import individual
from nslug.algorithms.GA.operators.mutators import ga_mutation,ga_delete,ga_insert
from nslug.algorithms.GA.operators.crossover_operators import single_point_crossover




In [ ]:
#X, y = load_gametes(2,10,0.1,X_y=True)
X, y = load_parkinson(X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)


In [2]:
X, y = load_gametes(3,1000,0.1,X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)

In [ ]:
FUNCTIONS = {
    'add': {'function': torch.add, 'arity': 2},
    'subtract': {'function': torch.sub, 'arity': 2},
    'multiply': {'function': torch.mul, 'arity': 2},
    'divide': {'function': protected_div, 'arity': 2}
}
CONSTANTS = {
    'constant_2': lambda _: torch.tensor(2.0),
    'constant_3': lambda _: torch.tensor(3.0),
    'constant_4': lambda _: torch.tensor(4.0),
    'constant_5': lambda _: torch.tensor(5.0),
    'constant__1': lambda _: torch.tensor(-1.0)
}
TERMINALS = get_terminals(X_train)

In [ ]:
TERMINALS

eCrossover

In [ ]:
from nslug.algorithms.GP.operators.crossover_operators import crossover_trees,ecrossover_trees

In [ ]:
tree=create_full_random_tree(4,FUNCTIONS, TERMINALS, CONSTANTS, p_c=0)
node_tree=len(list(flatten(tree)))
tree1=create_full_random_tree(4,FUNCTIONS, TERMINALS, CONSTANTS, p_c=0)
node_tree1=len(list(flatten(tree1)))

In [ ]:
used_terminal=used_terminals(tree,TERMINALS)
used_terminal

In [ ]:
keys_to_select = {'x0','x1','x4','x7','x8'}
TERMINALS1={k: TERMINALS[k] for k in keys_to_select if k in TERMINALS}
TERMINALS1

In [ ]:
aplica_ecrossover= ecrossover_trees(FUNCTIONS)

In [ ]:
off1,off2=aplica_ecrossover(X_train,tree,tree1,node_tree,node_tree1,TERMINALS1,used_terminal,TERMINALS)

In [ ]:
used_terminals(off1,TERMINALS)

In [ ]:
TERMINALS1

In [ ]:
used_terminals(off2,TERMINALS)

In [ ]:
used_terminal

In [ ]:
tree

In [ ]:
new_tree=fix_tree(tree,fix_terminals(X_train,used_terminal,TERMINALS1))
new_tree

GA indivuals, population and operators

In [ ]:
random_init(10,TERMINALS)

In [ ]:
individual.TERMINALS=TERMINALS

In [ ]:
population = Population_ga([individual(vec) for vec in random_init(10,TERMINALS)])

In [ ]:
ga_individual=population.population[0]

In [ ]:
ga_individual.encoder

In [ ]:
ga_individual.chromossome

In [ ]:
create_random_random_tree(6,FUNCTIONS,population.population[9].chromossome,CONSTANTS,0)

In [ ]:
ga_individual_delete=copy.deepcopy(ga_individual)
ga_individual_insert=copy.deepcopy(ga_individual)

In [ ]:
novo_dic={
  'x5': 5,
 'x7': 7,
 'x11': 11,
 'x12': 12,
 'x14': 14,
 'x17': 17,
 'x18': 18}

In [ ]:
novo_ga_delete=ga_delete(ga_individual_delete,novo_dic)

In [ ]:
novo_ga_insert=ga_insert(ga_individual_insert)

In [ ]:
ga_individual.chromossome

In [ ]:
novo_ga_delete.chromossome

In [ ]:
novo_ga_insert.chromossome

In [ ]:
tree_depth=math.ceil(math.log(len(ga_individual_insert.chromossome), 2))
terminal_list=create_custom_list(list(ga_individual_insert.chromossome.keys()),tree_depth)
terminal_list2=terminal_list.copy()

In [ ]:
tree_ga=create_full_random_tree_list_mode(tree_depth+1,FUNCTIONS,terminal_list2)

In [ ]:
dict(sorted(used_terminals(tree_ga,TERMINALS).items(), key=lambda item: item[1])) 

In [ ]:
tree_ga

In [ ]:
Tree.TERMINALS=TERMINALS
Tree.FUNCTIONS=FUNCTIONS
Tree.CONSTANTS=CONSTANTS
tree_ga_insert=copy.deepcopy(tree_ga)
tree_ga_insert=gp_insert(Tree(tree_ga_insert),novo_ga_insert.chromossome)

In [ ]:
tree_ga_insert.repr_

In [ ]:
dict(sorted(used_terminals(tree_ga_insert.repr_,TERMINALS).items(), key=lambda item: item[1]))  

In [ ]:
novo_ga_insert.chromossome

GP

In [ ]:
import time
start=time.time()
final_tree = gp(X_train=X_train, y_train=y_train,
                X_test=X_val, y_test=y_val,
                dataset_name='parkinsons', pop_size=100, n_iter=30, max_depth=None,minimization=False,verbose=1
               # ,TERMINALS={'x0':0}, init_depth=2
               )


end=time.time()

time=end-start

predictions = final_tree.predict(X_test)

In [ ]:
final_tree.repr_

In [ ]:
final_tree.fitness

In [ ]:
print(time)

In [ ]:
print(float(accuracy(y_true=y_test, y_pred=predictions)))

In [ ]:
from sklearn.metrics import accuracy_score
y_real= y_test.numpy().astype(bool)
y_previsto=(predictions > 0.5).int().numpy().astype(bool)

In [ ]:
accuracy_real = accuracy_score(y_real, y_previsto)
accuracy_real

In [76]:
ficheiros=(
(2,10,0.05),
(2,10,0.1),
(2,10,0.2),
(2,10,0.4),
(2,100,0.05),
(2,100,0.1),
(2,100,0.2),
(2,100,0.4),
(2,1000,0.05),
(2,1000,0.1),
(2,1000,0.2),
(2,1000,0.4),
(2,5000,0.05),
(2,5000,0.1),
(2,5000,0.2),
(2,5000,0.4),
(3,10,0.1),
(3,10,0.2),
(3,100,0.1),
(3,100,0.2),
(3,1000,0.1),
(3,1000,0.2)
)


In [3]:
ficheiros=(
(3,10,0.1),
(3,10,0.2),
(3,100,0.1),
(3,100,0.2),
(3,1000,0.1),
(3,1000,0.2)
)


In [17]:
ficheiros=(
(2,10,0.2),
(2,10,0.4),
(2,100,0.05),
(2,100,0.1),
(2,100,0.2),
(2,100,0.4),
(2,1000,0.2),
(2,1000,0.4)
)


In [71]:
X, y = load_gametes(ficheiros[0][0],ficheiros[0][1],ficheiros[0][2],X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)

import time
start=time.time()
final_tree = ga(X_train=X_train, y_train=y_train,
               # X_test=X_val, y_test=y_val,
                dataset_name='load_parkinson', 
                pop_size=100, n_iter=50, n_jobs=2,minimization=False)
end=time.time()

time=end-start
print(final_tree.fitness)

C:\Users\migue\anaconda3\lib\site-packages\nslug\main_ga.py:140: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")
C:\Users\migue\anaconda3\lib\site-packages\nslug\main_gp.py:143: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")
C:\Users\migue\anaconda3\lib\site-packages\nslug\main_gp.py:152: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


tensor(0.6317)


In [73]:
ficheiros=(
(2,10,0.05),
(2,10,0.1)
)
resultados=[]

In [74]:
for nr_epi, nr_var, her in ficheiros:
    X, y = load_gametes(nr_epi,nr_var,her,X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)
    
    import time
    start=time.time()
    final_tree = ga(X_train=X_train, y_train=y_train,
                   # X_test=X_val, y_test=y_val,
                    dataset_name='load_parkinson', 
                    pop_size=100, n_iter=50, n_jobs=2,minimization=False)
    end=time.time()

    time=end-start
    resultados.append((time,final_tree.fitness))
    


C:\Users\migue\anaconda3\lib\site-packages\nslug\main_ga.py:140: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")
C:\Users\migue\anaconda3\lib\site-packages\nslug\main_gp.py:143: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. test_elite has been set to False")
C:\Users\migue\anaconda3\lib\site-packages\nslug\main_gp.py:152: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")
C:\Users\migue\anaconda3\lib\site-packages\nslug\main_ga.py:140: UserWarning: If test_elite is True, a test dataset must be provided. test_elite has been set to False
  warnings.warn("If test_elite is True, a test dataset must be provided. tes

In [75]:
resultados

[(52.55325150489807, tensor(0.6317)), (39.869717597961426, tensor(0.6783))]

SLUG

In [ ]:
print(final_tree.fitness)
print(final_tree.model_repr_)
print(used_terminals(final_tree.model_repr_,TERMINALS))
print(time)

NSLUG

In [4]:
resultados=[]

versao standard

In [ ]:

for nr_epi, nr_var, her in ficheiros:
    X, y = load_gametes(nr_epi,nr_var,her,X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)
    
    import time
    start=time.time()

    final_tree = nslug(X_train=X_train, y_train=y_train,
                        X_test=X_val, y_test=y_val,
                        dataset_name='load_parkinson',verbose=False ,pop_size=150, n_iter=50, n_jobs=2,minimization=False, algorithm="standard")

    end=time.time()

    time=end-start
    print(final_tree.Tree.fitness)
    resultados.append((time,final_tree.Tree.fitness))
    



tensor(0.6092)


In [ ]:
resultados

In [ ]:
print(final_tree.Tree.fitness)
print(final_tree.Tree.repr_)
print(final_tree.individual.chromossome)
print(time)

In [ ]:
import time
start=time.time()

final_tree = nslug(X_train=X_train, y_train=y_train,
                    X_test=X_val, y_test=y_val,
                    dataset_name='load_parkinson', pop_size=250, n_iter=30, n_jobs=2,minimization=False, algorithm="split_populatition",pop_split=0.5)

end=time.time()

time=end-start

In [ ]:
print(final_tree.Tree.fitness)
print(final_tree.Tree.repr_)
print(final_tree.individual.chromossome)
print(time)

In [ ]:
from nslug.datasets.data_loader import load_gametes  

In [ ]:
X, y = load_gametes(2,10,0.1,X_y=True)

In [ ]:
X

In [ ]:
y